In [1]:
import edward as ed
from edward.models import Normal
from keras.layers import Input, Dense
from keras.regularizers import l2
from keras import backend as K
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import scale

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.style.use('seaborn-whitegrid')

/Users/zihaoxu/ENTER/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# ensure you are using TensorFlow as your Keras backend
sess = ed.get_session()
K.set_session(sess)

INIT_OP = tf.global_variables_initializer()

In [44]:
def change_datatype(df):
    int_cols = list(df.select_dtypes(include=['int']).columns)
    for col in int_cols:
        if ((np.max(df[col]) <= 127) and(np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767) and(np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647) and(np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)
    return df

# Loading Data

In [3]:
properties_df = pd.read_csv('data/properties.csv', low_memory=False)
transactions_df = pd.read_csv('data/transactions.csv')

In [6]:
properties_df = properties_df.rename(columns={
    'parcelid': 'id_parcel',
    'yearbuilt': 'build_year',
    'basementsqft': 'area_basement',
    'yardbuildingsqft17': 'area_patio',
    'yardbuildingsqft26': 'area_shed',
    'poolsizesum': 'area_pool',
    'lotsizesquarefeet': 'area_lot',
    'garagetotalsqft': 'area_garage',
    'finishedfloor1squarefeet': 'area_firstfloor_finished',
    'calculatedfinishedsquarefeet': 'area_total_calc',
    'finishedsquarefeet6': 'area_base',
    'finishedsquarefeet12': 'area_live_finished',
    'finishedsquarefeet13': 'area_liveperi_finished',
    'finishedsquarefeet15': 'area_total_finished',
    'finishedsquarefeet50': 'area_unknown',
    'unitcnt': 'num_unit',
    'numberofstories': 'num_story',
    'roomcnt': 'num_room',
    'bathroomcnt': 'num_bathroom',
    'bedroomcnt': 'num_bedroom',
    'calculatedbathnbr': 'num_bathroom_calc',
    'fullbathcnt': 'num_bath',
    'threequarterbathnbr': 'num_75_bath',
    'fireplacecnt': 'num_fireplace',
    'poolcnt': 'num_pool',
    'garagecarcnt': 'num_garage',
    'regionidcounty': 'region_county',
    'regionidcity': 'region_city',
    'regionidzip': 'region_zip',
    'regionidneighborhood': 'region_neighbor',
    'taxvaluedollarcnt': 'tax_total',
    'structuretaxvaluedollarcnt': 'tax_building',
    'landtaxvaluedollarcnt': 'tax_land',
    'taxamount': 'tax_property',
    'assessmentyear': 'tax_year',
    'taxdelinquencyflag': 'tax_delinquency',
    'taxdelinquencyyear': 'tax_delinquency_year',
    'propertyzoningdesc': 'zoning_property',
    'propertylandusetypeid': 'zoning_landuse',
    'propertycountylandusecode': 'zoning_landuse_county',
    'fireplaceflag': 'flag_fireplace',
    'hashottuborspa': 'flag_tub',
    'buildingqualitytypeid': 'quality',
    'buildingclasstypeid': 'framing',
    'typeconstructiontypeid': 'material',
    'decktypeid': 'deck',
    'storytypeid': 'story',
    'heatingorsystemtypeid': 'heating',
    'airconditioningtypeid': 'aircon',
    'architecturalstyletypeid': 'architectural_style'
})

transactions_df = transactions_df.rename(columns={
  'parcelid': 'id_parcel',
  'transactiondate': 'date'
})

In [7]:
print(properties_df.shape)
properties_df.head()

(2985217, 58)


,id_parcel,aircon,architectural_style,area_basement,num_bathroom,num_bedroom,framing,quality,num_bathroom_calc,deck,...,num_story,flag_fireplace,tax_building,tax_total,tax_year,tax_land,tax_property,tax_delinquency,tax_delinquency_year,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [8]:
print(transactions_df.shape)
transactions_df.head()

(90275, 3)


,id_parcel,logerror,date
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [9]:
df_train = transactions_df.merge(properties_df, how='left', on = 'id_parcel')
df_train.shape

(90275, 60)

In [16]:
keep_cols = df_train.columns[df_train.isnull().mean()<.10]
df_train = df_train[keep_cols]
df_train.shape

(90275, 26)

In [19]:
df_train = df_train.dropna()
df_train.shape

(83478, 26)

In [20]:
df_train.head()

,id_parcel,logerror,date,num_bathroom,num_bedroom,num_bathroom_calc,area_total_calc,area_live_finished,fips,num_bath,...,region_county,region_zip,num_room,build_year,tax_building,tax_total,tax_year,tax_land,tax_property,censustractandblock
0,11016594,0.0276,2016-01-01,2.0,3.0,2.0,1684.0,1684.0,6037.0,2.0,...,3101.0,96370.0,0.0,1959.0,122754.0,360170.0,2015.0,237416.0,6735.88,6.037107e+13
2,12098116,-0.0040,2016-01-01,3.0,2.0,3.0,2217.0,2217.0,6037.0,3.0,...,3101.0,96293.0,0.0,1940.0,61994.0,119906.0,2015.0,57912.0,11484.48,6.037464e+13
3,12643413,0.0218,2016-01-02,2.0,2.0,2.0,839.0,839.0,6037.0,2.0,...,3101.0,96222.0,0.0,1987.0,171518.0,244880.0,2015.0,73362.0,3048.74,6.037296e+13
4,14432541,-0.0050,2016-01-02,2.5,4.0,2.5,2283.0,2283.0,6059.0,2.0,...,1286.0,96961.0,8.0,1981.0,169574.0,434551.0,2015.0,264977.0,5488.96,6.059042e+13
5,11509835,-0.2705,2016-01-02,4.0,4.0,4.0,3067.0,3067.0,6037.0,4.0,...,3101.0,96109.0,0.0,1982.0,880650.0,2447951.0,2015.0,1567301.0,27126.57,6.037621e+13


In [40]:
corr_rank = df_train.corr()['logerror'].abs().sort_values(ascending = False).head(11)[1:]
corr_rank

area_total_calc        0.042789
area_live_finished     0.042789
num_bathroom           0.035256
num_bathroom_calc      0.035256
num_bath               0.034803
num_bedroom            0.032429
tax_building           0.023352
build_year             0.020811
region_zip             0.009172
censustractandblock    0.009088
Name: logerror, dtype: float64

In [41]:
most_predictive_cols = list(corr_rank.index)
print(most_predictive_cols)

['area_total_calc', 'area_live_finished', 'num_bathroom', 'num_bathroom_calc', 'num_bath', 'num_bedroom', 'tax_building', 'build_year', 'region_zip', 'censustractandblock']


In [45]:
coord_cols = ['latitude', 'longitude']

change_datatype(df_train).to_csv('data/df_all_col.csv', index = None)
change_datatype(df_train[most_predictive_cols+['logerror']+coord_cols]).to_csv('data/df_imp_col.csv', index = None)